In [1]:
using JuMP
#using Gurobi
m = Model() #solver = GurobiSolver

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is default solver

In [2]:
# Problem definition
nctrl = 10; # number of control points
nt = 5; # number of time steps
r = 1; # radius of the motor
circ = 2*pi*r; # maximum circumference of the motor
circProfile = 2*pi*r*ones(nt);
regrate = 0.05;

In [3]:
#@variable(m, 0 <= ctrlRadius[1:nctrl,1:nt] <= r)
#@variable(m, 0 <= ctrlTheta[1:nctrl,1:nt] <= 2*pi)
# JuMP does not support complex numbers
@variable(m, 0 <= x[1:nctrl,1:nt] <= r)
@variable(m, 0 <= y[1:nctrl,1:nt] <= r)
@variable(m, nx[1:nctrl,1:nt] >= 0)
@variable(m, ny[1:nctrl,1:nt] >= 0)
@variable(m, l[1:nctrl,1:nt] >= 0)
@variable(m, circ[1:nt] >= 0)

5-element Array{JuMP.Variable,1}:
 circ[1]
 circ[2]
 circ[3]
 circ[4]
 circ[5]

In [4]:
for i in range(1,nctrl)
    for j in 1:nt
        # Geometry bounding
        @constraint(m, x[i,j]^2 + y[i,j]^2 <= r^2)
    end
end

# Normal vector and length (of LHS edge of current point) calculation
for j in 1:nt
    @NLconstraint(m, nx[1,j] == -(y[1,j]-y[nctrl,j])/l[nctrl,j])
    @NLconstraint(m, ny[1,j] == (x[1,j]-x[nctrl,j])/l[nctrl,j])
    @NLconstraint(m, l[1,j]^2 == (x[1,j] - x[nctrl, j])^2 + (y[1,j] - y[nctrl, j])^2)
    for i in 2:nctrl
        @NLconstraint(m, nx[i,j] == -(y[i,j]-y[i-1,j])/l[i-1,j])
        @NLconstraint(m, ny[i,j] == (x[i,j]-x[i-1,j])/l[i-1,j])
    end
end

# Motion of the flame front
for j in 2:nt
    @NLconstraint(m, x[nctrl,j] == x[nctrl,j-1] + 0.5*nx[nctrl,j-1]*regrate + 0.5*nx[1,j-1]*regrate)
    @NLconstraint(m, y[nctrl,j] == y[nctrl,j-1] + 0.5*ny[nctrl,j-1]*regrate + 0.5*ny[1,j-1]*regrate)
    for i in 1:nctrl-1
    @NLconstraint(m, x[i,j] == x[i,j-1] + nx[i,j-1]*regrate + nx[i+1,j-1]*regrate)
    @NLconstraint(m, y[i,j] == y[i,j-1] + ny[i,j-1]*regrate + ny[i+1,j-1]*regrate)
    end
end
    


In [5]:
@objective(m, Min, sum((circProfile[i]-circ[i])^2 for i in nt))

:Min

In [6]:
solution = solve(m)

LoadError: ReadOnlyMemoryError()